# Analyze movement based on Accelerometer Data

In [ ]:
# Importing Python and external packages
import os
import sys
import importlib
import json
import csv
from dataclasses import dataclass, field, fields
from collections import namedtuple
from itertools import compress
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from  matplotlib import __version__ as plt_version
from scipy import signal, stats

# import datetime as dt
# #mne
# import mne_bids
# import mne


In [ ]:
def get_project_path_in_notebook(
    subfolder: str = '',
):
    """
    Finds path of projectfolder from Notebook.
    Start running this once to correctly find
    other modules/functions
    """
    path = os.getcwd()

    while path[-20:] != 'dyskinesia_neurophys':

        path = os.path.dirname(path)
    
    return path

In [ ]:
# define local storage directories
projectpath = get_project_path_in_notebook()
codepath = os.path.join(projectpath, 'code')
figpath = os.path.join(projectpath, 'figures')
datapath = os.path.join(projectpath, 'data')


In [ ]:
os.chdir(codepath)

# own functions
import lfpecog_features.moveDetection_preprocess as movePrep
import lfpecog_features.moveDetection_run as run_tap_detect
import lfpecog_features.moveDetection_pausedTapFinder as findTap

import lfpecog_analysis.load_SSD_features as load_ssd_fts

import utils.utils_fileManagement as utilsFiles
from utils.utils_fileManagement import (get_project_path,
                                        load_class_pickle,
                                        save_class_pickle,
                                        mergedData,
                                        correct_acc_class)
import lfpecog_preproc.preproc_import_scores_annotations as importClin
import lfpecog_plotting.plotHelpers as plotHelp

In [ ]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('matplotlib', plt_version)

### 1a. Load pickled acc-data



In [ ]:
import lfpecog_analysis.movement_psd_analysis as movePSD

In [ ]:

data_version = 'v4.0'
main_data_path = os.path.join(get_project_path('data'),
                              'merged_sub_data',
                              data_version)

SUBS = load_ssd_fts.get_avail_ssd_subs(DATA_VERSION=data_version)
print(SUBS)

In [ ]:


for sub in SUBS:

    if sub in ['010']: continue
    
    sub_data_path = os.path.join(main_data_path,
                                 f'sub-{sub}')
    for side in ['left', 'right']:
        if side == 'right': continue

        fname = (f'{sub}_mergedData_{data_version}'
                f'_acc_{side}.P')

        # load Acc-detected movement labels
        acc = load_class_pickle(os.path.join(sub_data_path, fname))
        print(f'\t...sub-{sub}: loaded {side} Acc-Pickle')
        acc = correct_acc_class(acc)

        movePSD.plot_overview_tap_detection(acc, SAVE_FIG=True)


### 1b. Load SSD data

In [ ]:
import lfpecog_features.get_ssd_data as ssd
import lfpecog_analysis.get_SSD_timefreqs as ssd_TimeFreq

In [ ]:
# get windowed bands of different dtypes per sub
# importlib.reload(ssd)

sub = '016'

# # call from feats_extract_multivar.py
# ssd_017 = ssd.get_subject_SSDs(
#     sub=sub,
#     incl_stn=True,
#     incl_ecog=True,
#     ft_setting_fname='ftExtr_spectral_v4.json',)

# call from feats_extract_multivar.py
ssd_sub = ssd.get_subject_SSDs(
    sub=sub,
    incl_stn=True,
    incl_ecog=False,
    ft_setting_fname='ftExtr_spectral_v4.json',)

### 2. Explore ACC-activity analysis

In [ ]:
sub = '016'

acc = {}

sub_data_path = os.path.join(main_data_path,
                                 f'sub-{sub}')
fname = (f'{sub}_mergedData_{data_version}'
            f'_acc_{side}.P')

# load Acc-detected movement labels
for side in ['left', 'right']:
    fname = (f'{sub}_mergedData_{data_version}'
                f'_acc_{side}.P')
    acc[side] = load_class_pickle(os.path.join(sub_data_path, fname))
    # fname = (f'{sub}_mergedData_{data_version}'
    #             f'_acc_right.P')
    # acc_r = load_class_pickle(os.path.join(sub_data_path, fname))

In [ ]:
acc['left'].colnames

In [ ]:
def create_tap_kernel(
    duration_sec=2, Fs=512,
):
    # kernel for 017
    kernel_pattern = np.array([0, 0, 1, -1, 2, -2, 2, -1, 1, 0, 0])

    pad = int((Fs * duration_sec) / len(kernel_pattern))
    
    kernel = np.array(
        [np.linspace(kernel_pattern[i],
                     kernel_pattern[i+1], pad)
         for i in np.arange(len(kernel_pattern) - 1)]
    ).ravel()

    return kernel

In [ ]:
import lfpecog_features.feats_helper_funcs as ftHelpers

In [ ]:
def custom_tap_finding_017(acc_l_017):
    """
    custom needed due to restless legs and
    very slow hand raising

    results in indices and times of taps
    with left hand only
    Tapping movement starts circa two seconds
    before and one second after times

    Inputs: 
        - acc: acc class from pickle acc 017
    """
    fs = int(acc_l_017.fs)

    d_og = acc_l_017.data[:, 1]  # 1 because of ACC_L_X
    t_og = acc_l_017.times
    idx_og = np.arange(len(d_og))
    task = acc_l_017.data[:, 4]  # 4 is task column

    sel = task == 'tap'

    d = d_og[sel]
    t = t_og[sel]
    idx = idx_og[sel]

    THR=.5e-7
    peak_idx, peak_props = signal.find_peaks(d, height=THR, distance=int(fs*5))

    peak_t = t[peak_idx]
    peak_i = idx[peak_idx]

    # create boolean positive during tap
    tap_bool = np.zeros(len(d_og))
    for i in peak_i:
        try:
            if t_og[i+fs] - t_og[i-(2*fs)] > 3:
                tap_bool[i - (2*fs):i] = 1
            else: tap_bool[i - (2*fs):i+fs] = 1
        except IndexError:
            tap_bool[i - (2*fs):i] = 1


    # plt.scatter(t[peak_idx], np.ones(len(peak_idx)) * 1e-7,
    #             color='red', s=10)

    # plt.plot(t_og, d_og, color='green', alpha=.5)

    # plt.xlim(14, 15)
    # plt.show()

    return peak_i, peak_t, tap_bool

In [ ]:
def select_taps_in_data(
    neural_data, neural_times, tap_bool, tap_bool_times,
    margin_around_tap_sec=1, neural_fs=2048, return_nontap=False,
    move_excl_bool=False, move_excl_times=False,
):
    assert len(neural_data) == len(neural_times), (
        'length neural data and times do not match'
    )
    assert len(tap_bool) == len(tap_bool_times), (
        'length tap_bool and times do not match'
    )
    # convert to arrays
    if isinstance(neural_data, list): neural_data = np.array(neural_data)
    if isinstance(neural_times, list): neural_times = np.array(neural_times)

    # find tap start end indeices
    tap_df = np.diff(tap_bool.astype(int))
    tap_starts = np.where(tap_df == 1)[0]  # gives indices
    tap_ends = np.where(tap_df == -1)[0]
    # convert indices into time (dopa_time seconds)
    tap_starts = tap_bool_times[tap_starts]
    tap_ends = tap_bool_times[tap_ends]

    # create boolean arrays to adjust
    data_tap_bool = np.zeros_like(neural_data)  # change to positive during taps
    if return_nontap:
        data_notap_bool = np.ones_like(neural_data)  # change to negative during tap plus pad
        pad_idx = int(margin_around_tap_sec * neural_fs)

    for t1, t2 in zip(tap_starts, tap_ends):
        # find indices in neural data for tap start and end
        i1 = np.argmin(abs(neural_times - t1))
        i2 = np.argmin(abs(neural_times - t2))
        # set selected time window to positive in bool
        data_tap_bool[i1:i2] = 1

        if return_nontap:
            data_notap_bool[i1 - pad_idx:i2 + pad_idx] = 0

    if return_nontap and not isinstance(move_excl_bool, bool):
         # find tap start end indeices for CONTRA LAT TAPPING
        tap_df = np.diff(move_excl_bool.astype(int))
        tap_starts = np.where(tap_df == 1)[0]  # gives indices
        tap_ends = np.where(tap_df == -1)[0]
        # convert indices into time (dopa_time seconds)
        tap_starts = move_excl_times[tap_starts]
        tap_ends = move_excl_times[tap_ends]

        # set no_tap_bool to negative for contra-taps
        for t1, t2 in zip(tap_starts, tap_ends):
            # find indices in neural data for tap start and end
            i1 = np.argmin(abs(neural_times - t1))
            i2 = np.argmin(abs(neural_times - t2))
            # set selected time window to negative in bool
            data_notap_bool[i1 - pad_idx:i2 + pad_idx] = 0



    # select neural data based on bool
    tap_data = neural_data[data_tap_bool.astype(bool)]

    if return_nontap:
        no_tap_data = neural_data[data_notap_bool.astype(bool)]

    if not return_nontap: return tap_data
    
    if return_nontap: return tap_data, no_tap_data 

In [ ]:
def excl_specific_task(
    data_arr, data_times, task_arr, task_times,
    task_to_excl
):
    # convert to arrays
    if isinstance(data_times, list): data_times = np.array(data_times)
    if isinstance(task_times, list): task_times = np.array(task_times)
    if isinstance(task_arr, list): task_arr = np.array(task_arr)
    
    task_changes = np.diff((task_arr == task_to_excl).astype(int)) 
    idx_task_start = np.where(task_changes== 1)[0]
    idx_task_end = np.where(task_changes == -1)[0]

    t_task_start = task_times[idx_task_start]
    t_task_end = task_times[idx_task_end]

    # set true to keep, false to exclude
    incl_bool = np.ones_like(data_arr)

    for t1, t2 in zip(t_task_start, t_task_end):
        # find indices in neural data for tap start and end
        i1 = np.argmin(abs(data_times - t1))
        i2 = np.argmin(abs(data_times - t2))

        incl_bool[i1:i2] = 0

    # select neural data based on bool
    data_arr = data_arr[incl_bool.astype(bool)]
    data_times = data_times[incl_bool.astype(bool)]


    return data_arr, data_times

In [ ]:
ssd_sub.lfp_left.delta.shape

In [ ]:
importlib.reload(ssd_TimeFreq)

lfp_side = 'lfp_right'
acc_side = 'left'
acc_contra_side = 'right'

# create continuous data timeseries for ssd'd signal
cont_sig, sig_times, _ = ssd_TimeFreq.get_cont_ssd_arr(
    subSourceSSD=getattr(ssd_sub, lfp_side),
    bw='gamma',
    winLen_sec=ssd_sub.settings['WIN_LEN_sec'],)

# find corresponding tap-times
if ssd_sub.sub == '017':
    peak_idx, peak_t, tap_bool = custom_tap_finding_017(acc[acc_side])
else:
    i_col_sidetap = [i for i, c in enumerate(acc[acc_side].colnames)
                     if c == f'{acc_side}_tap'][0]
    tap_bool = acc[acc_side].data[:, i_col_sidetap]
    
    # to exclude contralat taps from no-tap (rest)
    i_col_contratap = [i for i, c in enumerate(acc[acc_side].colnames)
                     if c == f'{acc_contra_side}_tap'][0]
    contra_tap_bool = acc[acc_side].data[:, i_col_contratap]

    i_move = [i for i, c in enumerate(acc[acc_side].colnames)
                     if c == f'{acc_contra_side}_move'][0]
    move_bool = np.logical_or(contra_tap_bool, acc[acc_side].data[:, i_move])
    
    i_move = [i for i, c in enumerate(acc[acc_side].colnames)
                     if c == f'{acc_side}_move'][0]
    move_bool = np.logical_or(move_bool, acc[acc_side].data[:, i_move])

# exclude free-tasks for tap vs rest analysis
i_col_task = [i for i, c in enumerate(acc[acc_side].colnames) if c == 'task'][0]
cont_sig, sig_times = excl_specific_task(
    data_arr=cont_sig,
    data_times=sig_times,
    task_arr=acc[acc_side].data[:, i_col_task],
    task_times=acc[acc_side].times,
    task_to_excl='free'
)

# split data in tap vs no-tap
tap_data = select_taps_in_data(
    neural_data=cont_sig,
    neural_times=sig_times,
    tap_bool=tap_bool,
    tap_bool_times=acc[acc_side].times,
    return_nontap=False,)

# split data in tap vs no-tap
cont_sig_rest, sig_times_rest = excl_specific_task(
    data_arr=cont_sig,
    data_times=sig_times,
    task_arr=acc[acc_side].data[:, i_col_task],
    task_times=acc[acc_side].times,
    task_to_excl='tap'
)

_, no_tap_data = select_taps_in_data(
    neural_data=cont_sig_rest,
    neural_times=sig_times_rest,
    tap_bool=tap_bool,
    tap_bool_times=acc[acc_side].times,
    return_nontap=True,
    move_excl_bool=move_bool,
    move_excl_times=acc[acc_side].times,
    neural_fs=getattr(ssd_sub, lfp_side).fs,
    margin_around_tap_sec=2,
)




In [ ]:
import lfpecog_analysis.movement_psd_analysis as movePSD

In [ ]:
importlib.reload(movePSD)

for sub in SUBS:
    if sub in ['016', '101', '010', '017']: continue
    try:
        movePSD.create_sub_movement_psds(sub=sub)
    except:
        print(f'sub {sub} failed and skipped')
        continue

In [ ]:
n_wins = len(no_tap_data) // 2048
no_tap_data_wins = no_tap_data[:n_wins*2048].reshape(n_wins, 2048)

print(no_tap_data_wins.shape)

f, psx = signal.welch(x=no_tap_data_wins, fs=ssd_sub.lfp_left.fs,
             nperseg=ssd_sub.lfp_left.fs // 1, axis=1)

alpha_wins = np.linspace(0.2, 1, num=psx.shape[0])

for r, ps_row in enumerate(psx):
    plt.plot(f, ps_row, color='purple', alpha=alpha_wins[r])

plt.xlim(4, 98)

plt.show()

In [ ]:
n_wins = len(tap_data) // 2048
tap_data_wins = tap_data[:n_wins*2048].reshape(n_wins, 2048)

print(tap_data_wins.shape)

f, psx = signal.welch(x=tap_data_wins, fs=ssd_sub.lfp_left.fs,
             nperseg=ssd_sub.lfp_left.fs // 1, axis=1)

alpha_wins = np.linspace(0.2, 1, num=psx.shape[0])

for r, ps_row in enumerate(psx[:50]):
    plt.plot(f, ps_row, color='purple', alpha=alpha_wins[r])

plt.xlim(4, 98)

plt.show()

In [ ]:
print(tap_data.shape, no_tap_data.shape)


f, psx = signal.welch(x=tap_data, fs=ssd_sub.lfp_left.fs,
             nperseg=ssd_sub.lfp_left.fs // 1,)

plt.plot(f, psx, label='taps')

f, psx = signal.welch(x=no_tap_data, fs=ssd_sub.lfp_left.fs,
             nperseg=ssd_sub.lfp_left.fs // 1,)

plt.plot(f, psx, label='no taps')

plt.xlim(4, 98)

plt.legend(loc='upper left')

plt.show()

In [ ]:
print(len(tap_data) / 2048, len(no_tap_data) / 2048)


f, psx = signal.welch(x=tap_data, fs=ssd_017.lfp_left.fs,
             nperseg=ssd_017.lfp_left.fs // 1,)

plt.plot(f, psx, label='taps')

f, psx = signal.welch(x=no_tap_data, fs=ssd_017.lfp_left.fs,
             nperseg=ssd_017.lfp_left.fs // 1,)

plt.plot(f, psx, label='no taps')

plt.xlim(4, 98)

plt.legend(loc='upper left')

plt.show()

In [ ]:
def unpack_list_of_lists(lists_to_unpack):

    new_list = [i for j in lists_to_unpack for i in j]

    return new_list

#### 1b) Import processed dataclass per subject, optionally merge ACC-data into EPHYS-df

In [ ]:
# # Import subjectData Classes with alligned preprocessed Data
# importlib.reload(read_data)
# importlib.reload(add_moveStates)
# importlib.reload(findTap)

# incl_accStates = True

# subData = {}

# for sub in ['012', '014']:

#     print(f'start {sub}')
#     # if sub == '008': continue

#     subData[sub] = read_data.subjectData(
#         sub=sub,
#         data_version='v2.3',
#         project_path=projectpath,
#     )

#     if incl_accStates:
        
#         accStates = run_tap_detect.runTapDetection(subData[sub])

#         for group in subData[sub].dtypes:

#             if 'lfp' or 'ecog' in group:

#                 print(f'adding acc-states for {sub}: {group}')
#                 newdf = add_moveStates.add_detected_acc_states(
#                     df=getattr(subData[sub], group).data,
#                     detectedMoves=accStates,
#                 )
#                 getattr(subData[sub], group).data = newdf

### 2) Develop and Visualise Movement State Detection


#### Run single Acc-State Detections


In [ ]:
importlib.reload(run_tap_detect)
importlib.reload(movePrep)
importlib.reload(findTap)

taplists = {}
for sub in ['012',  '014']:  # '008', '013',
    print(sub)
    taplists[sub] = run_tap_detect.runTapDetection(subData[sub])

#### Visualise Performance of Tap/Move-detection

In [ ]:

fonts=20

for sub in list(subData.keys()):

    for x0, x1 in zip(
        # [9, 42],
        # [10, 43]
        [5, 37,],
        [15, 42]
    ):

        fig, axes = plt.subplots(2, 1, figsize=(16, 8))

        for s, side in enumerate(['left', 'right']):

            acc_df = getattr(subData[sub], f'acc_{side}').data  # per side
            fs = getattr(subData[sub], f'acc_{side}').fs

            ax = movePrep.find_main_axis(
                acc_df.iloc[:, 1:4].values
            )
            svm = movePrep.signalvectormagn(
                acc_df.iloc[:, 1:4].values
            )

            axes[s].plot(
                acc_df['dopa_time'] / 60,
                acc_df.iloc[:, ax + 1],
                alpha=.4, label='uni-axis'
            )
            axes[s].plot(
                acc_df['dopa_time'] / 60,
                movePrep.signalvectormagn(
                    acc_df.iloc[:, 1:4].values
                ), alpha=.4, label='svm', c='r', ls='dotted',
            )

            axes[s].scatter(
                np.array([l[0] for l in taplists2[sub][f'{side}_tap_t']]) / 60,
                [.65e-6] * len(taplists2[sub][f'{side}_tap_t']),
                s=50, color='g', label='tap-start',
            )
            axes[s].scatter(
                np.array([l[-1] for l in taplists2[sub][f'{side}_tap_t']]) / 60,
                [.6e-6] * len(taplists2[sub][f'{side}_tap_t']),
                s=50, color='r', label='tap-end'
            )
            axes[s].scatter(
                np.array([m[0] for m in taplists2[sub][f'{side}_move_t']]) / 60,
                [.55e-6] * len(taplists2[sub][f'{side}_move_t']),
                s=50, color='orange', label='move-start'
            )
            axes[s].scatter(
                np.array([m[1] for m in taplists2[sub][f'{side}_move_t']]) / 60,
                [.5e-6] * len(taplists2[sub][f'{side}_move_t']),
                s=50, color='purple', label='move-end'
            )

            axes[s].set_xlim(x0, x1)
            axes[s].set_ylim(-1e-6, 1e-6)
            axes[s].set_ylabel(
                f'Acceleration\n{side.upper()}'
                    '\n(g, m/s/s)',
                size=fonts
            )
            axes[s].tick_params(labelsize=fonts - 4)

        axes[s].set_xlabel('Time (minutes to L-Dopa intake)', size=fonts)


        plt.suptitle(
            f'Subject {sub} -  bilateral '
            'Movement detection',
            x=.1, y=.96, ha='left',
            size=fonts+4
        )
        # remove duplicate legend labels
        handles, labels = plotHelp.remove_duplicate_legend(
            plt.gca().get_legend_handles_labels()
        )

        fig.legend(
            handles, labels,
            frameon=False, fontsize=fonts - 4, ncol=3,
            loc='center left', bbox_to_anchor = [.55, .95])
        
        plt.tight_layout()

        # plt.savefig(os.path.join(
        #     fig_dir, 'tapping_detection',
        #     f'sub{sub}_moveDetect_newBorders_min{x0}_{x1}'
        # ), dpi=150, facecolor='w',)

        plt.close()

### VIsualisation Features

#### Manual Video-Movement Annotation

In [ ]:
# annotate observed taps in video, integers are seconds
# from video start
# CAVE: orignal Video-starttime is hour in front
# values here are corrected one hour backwards (12->11)
# Still the video-time seems to be +/- 3 minutes behind
# the acc-neurophys-time
taptimes_video = {
    'SelfpacedHandTapL_StimOffDopa15': {
        'left': [
            1154, 1160, 1164, 1169, 1174, 1180, 1185,
            1191, 1195, 1201, 1207, 1226, 1238, 1243,
            1248, 1253
        ],
        'right':[
            1260, 1264, 1270, 1275, 1281, 1290, 1298,
            1303, 1308, 1313, 1320, 1324, 1332, 1339,
            1345
        ],
        'starttime': '2021-11-15 11:29' 
        # original 11:26 but corrected for mismatching times
    },
    'SelfpacedHandTapL_StimOffDopa35': {
        'left': [
            878, 883, 888, 893, 898, 903, 908, 915,
            919, 926, 933, 946, 951, 957, 963, 970,
            976, 984
        ],
        'right':[
            1000, 1006, 1011, 1017, 1024, 1028, 1035,
            1041, 1054, 1060, 1081, 1089, 1097, 1103,
            1108
        ],
        'starttime': '2021-11-15 11:51:01'
        # original 11:48 but corrected for mismatching times
    },
    'SelfpacedHandTapL_StimOffDopa60': {
        'left': [
            641, 647, 652, 657, 662, 666, 671, 676,
            682, 686, 692, 697, 703, 708, 713, 719,
        ],
        'right':[
            726, 730, 734, 740, 746, 751, 756, 761,
            768, 773, 779, 784, 792, 797, 801
        ],
        'starttime': '2021-11-15 12:22:59'  # video S1290007
        # original 12:20 but corrected for mismatching times
    }
}
# add timedelta in seconds to start-time of video
# CAVE videostarttime is on MINUTE-RESOLUTION
# thus will be SECONDS OF FROM ACC-TIMESTAMP
for run in taptimes_video:
    starttime = pd.Timestamp(taptimes_video[run]['starttime'])
    for side in ['left', 'right']:
        taptimes_video[run][f'{side}_stamps'] = []
        for sec in taptimes_video[run][side]:
            delta = pd.Timedelta(sec, 'sec')
            taptimes_video[run][f'{side}_stamps'].append(
                starttime + delta)

# Save annotation dict
# deriv_dir = os.path.join(
#     projectpath, 'data', 'analysis_derivatives', 'sub-008', 'taps'
# )
# dict_name = f'008_video_ann_tapruns'
# np.save(os.path.join(deriv_dir, dict_name),
#         taptimes_video)

# # Load annotation dict
# video_taps = np.load(os.path.join(deriv_dir, f'{dict_name}.npy'),
#                      allow_pickle=True).item()


In [ ]:
sub = '008'
annot_dir = os.path.join(projectpath, f'data/analysis_derivatives/sub-{sub}/taps')
testannot = np.load(os.path.join(annot_dir, f'{sub}_video_ann_tapruns.npy'),
    allow_pickle=True).item()
# testannot


#### Automated Acc-based Tap detection incl. plotting

In [ ]:
run = 'SelfpacedHandTapL_StimOffDopa35'
sides = ['left', 'right']
tap_fig_dir = os.path.join(projectpath, 'figures/tapping_detection')

In [ ]:
importlib.reload(tap_run)

restTest, tapTest, _ = tap_run.runTapDetection(
    task='paused', fs=SUB08.runs[run].acc_right_Fs,
    leftxyz=[accleft[run][0, :], accleft[run][1, :],
    accleft[run][2, :]], rightxyz=[accright[run][0, :],
    accright[run][1, :], accright[run][2, :]],
)

In [ ]:
### Main function to perform TapDetection functions
    ## make new main-function

importlib.reload(Tap2)

# accright, accleft = {}, {}
# restDict, tapDict, moveDict = {}, {}, {}

for run in SUB08.runs_incl[1:]:
    print(f'Start RUN: {run}')
    fs=SUB08.runs[run].acc_right_Fs
    accright[run] = SUB08.runs[run].acc_right_arr[1:, :]
    accleft[run] = SUB08.runs[run].acc_left_arr[1:, :]

    restDict[run], tapDict[run], moveDict[run] = runTapDetection(
        task='paused', fs=SUB08.runs[run].acc_right_Fs,
        leftxyz=[accleft[run][0, :], accleft[run][1, :],
        accleft[run][2, :]], rightxyz=[accright[run][0, :],
        accright[run][1, :], accright[run][2, :]],
    )


In [ ]:
### (function to be) get runstartTimes (include in RUN CLASS (real and dopa times))
deriv_dir = os.path.join(projectpath, 'data/analysis_derivatives')
runTimes = pd.read_excel(
    os.path.join(deriv_dir, f'sub-{sub}', f'sub{sub}_runsInfo.xlsx'))

runStartTimes = {}
for run in SUB08.runs_incl:
    for row in np.arange(runTimes.shape[0]):
        if run[-6:] in runTimes.iloc[row]['filename']:
            t = runTimes.iloc[row]['acq_time']
            runStartTimes[run] = pd.Timestamp(t)

In [ ]:
### Import and process video-annotated tap-times ###

## (function to be) Convert video-annotated tap-stamps
## into seconds after tmsi-recording

# import annotated taptimes (function to be?)
sub = '008'
annot_dir = os.path.join(projectpath, f'data/analysis_derivatives/sub-{sub}/taps')
tapAnnot = np.load(os.path.join(annot_dir, f'{sub}_video_ann_tapruns.npy'),
    allow_pickle=True).item()

ann_times = {}
tapsAnnot_in_runSecs = {}

for run in tapDict.keys():
    if 'Tap' not in run: continue

    # real-time timestamps of video-annotated taps
    dt_string = '%Y-%m-%d %H:%M:%S'
    t0 = tapAnnot[run]['starttime']
    try:
        t0 = dt.datetime.strptime(t0, dt_string)
    except ValueError:
        t0 = dt.datetime.strptime(t0, '%Y-%m-%d %H:%M')

    ann_times[run] = {}
    tapsAnnot_in_runSecs[run] = {}

    for side in sides:
        ann_times[run][side] = [t0 + dt.timedelta(
            seconds=t) for t in tapAnnot[run][side]]
    # minus video start-time -> time after video-start
        tapsAnnot_in_runSecs[run][side] = [
            (t - runStartTimes[run]).total_seconds(
            ) for t in ann_times[run][side]
        ]


In [ ]:
### TO DO:
# - WHY IS check_Polarity turning the signal falsely?
# - Why is tapdetection from TapDopa15 Right empty? 
    # plot incl peaks and thresholds etc

# - include plotting of OTHER MVOEMENTS
# - split py function for tapping in several py docs!!

In [ ]:
### Create plotting functions for tapping

from matplotlib.gridspec import GridSpec

run = list(tapsAnnot_in_runSecs.keys())[2]

print(f'Plot Figure for run {run}')
# specify x-coordinates of zoomed-in subplots
ax1_x1, ax1_x2 = 72, 80
ax2_x1, ax2_x2 = 150, 160
plot_fname=f'TapDetect_08_{run}'
if ax1_x1: plot_fname += f'_zooms_{ax1_x1}_{ax2_x1}'
plot_fdir=tap_fig_dir

plot_params = {
        'left': {
            'color': 'b',
            'alpha': .6
        },
        'right': {
            'color': 'g',
            'alpha': .6
        },
        'rest': {
            'color': 'orange',
            'alpha': .3
        },
        'left_up': {
            'color': 'dodgerblue',
            'alpha': .5
        },
        'left_down': {
            'color': 'cyan',
            'alpha': .5
        },
        'left_other': {
            'facecolor': 'white',
            'edgecolor': 'dodgerblue',
            'alpha': .8,
            'hatch': '//'
        },
        'right_up': {
            'color': 'forestgreen',
            'alpha': .5
        },
        'right_down': {
            'color': 'lime',
            'alpha': .5
        },
        'right_other': {
            'facecolor': 'white',
            'edgecolor': 'forestgreen',
            'alpha': .8,
            'hatch': '//'
        }
    }

fig = plt.figure(constrained_layout=True, figsize=(16, 12))
gs = GridSpec(3, 9, figure=fig)

ax0 = fig.add_subplot(gs[:2, :])
ax1 = fig.add_subplot(gs[2, :4])
ax2 = fig.add_subplot(gs[2, 5:], sharey=ax1)
plt.setp(ax2.get_yticklabels(), visible=False)

for ax in [ax0, ax1, ax2]:
    # Plotting traces
    ax.plot(
        np.arange(0, accleft[run][1, :].shape[0]) / fs,
        accleft[run][1, :], **plot_params['left'],
        label='Left ACC-trace',
    )
    ax.plot(
        np.arange(0, accright[run][1, :].shape[0]) / fs,
        accright[run][1, :], **plot_params['right'],
        label='Right ACC-trace',
    )
    # Plot bilateral Rest-epochs
    for d in restDict[run]:
        ax.fill_betweenx(label='Bilat. Rest detected',
            y=[-1e-6, 1e-6], x1=restDict[run][d][0],
            x2=restDict[run][d][1],
            **plot_params['rest'],)

    # Plot Tap and other-movem. epochs
    for side in ['left', 'right']:
        ## plot Taps
        for t, tList in enumerate(tapDict[run][side]):
            ax.fill_betweenx(
                y=[-3e-6, 3e-6],
                x1=tList[0], x2=tList[2],
                label=f'{side} Up detected',
                **plot_params[f'{side}_up'],
            )
            # ax.scatter(tList[1], )  # plot fastest point
            ax.fill_betweenx(
                y=[-3e-6, 3e-6],
                x1=tList[3], x2=tList[4],
                label=f'{side} Down detected',
                **plot_params[f'{side}_down'],
            )
        
        ## Plot Other Movement's
        for t, tList in enumerate(moveDict[run][side]):
            ax.fill_betweenx(
                y=[-1e-6, 1e-6],
                x1=tList[0], x2=tList[1],
                label=f'{side} Other Mov. detected',
                **plot_params[f'{side}_other'],
            )
        
        ## Plot Accuracy circles based on video-annotations
        vid_circle_pos = tapsAnnot_in_runSecs[run][side]
        acc_circle_pos = [t[0] for t in tapDict[run][side]]
        ax0.scatter(
            vid_circle_pos, [3.5e-6] * len(vid_circle_pos),
            s=150, edgecolor='k', facecolor='w', lw=2,
            label='Video-annotated Tap', **plot_params['left'],
        )
        ax0.scatter(
            acc_circle_pos, [3.5e-6] * len(acc_circle_pos),
            s=120, color=plot_params[side]['color'], alpha=.3,
            label=f'Acc-classified Tap {side}'
        )
        tapAcc_handles, tapAcc_labels = ax0.get_legend_handles_labels()
        tapAcc_handles = tapAcc_handles[-3:]
        tapAcc_labels = tapAcc_labels[-3:]
        ax0.legend(tapAcc_handles, tapAcc_labels,
            fontsize=16, ncol=3, frameon=False,
            loc='upper center', bbox_to_anchor=(.5, .99))

        # plot findpeak dots to optimize algorithm
        # posPeaks = signal.find_peaks(
        #     accleft[run][1, :],
        #     height=np.max(accleft[1, :]) * .5,
        #     distance=fs,  # 1 s
        # )
        # ax.scatter(posPeaks[0] / fs, posPeaks[1]['peak_heights'], c='k')
        # posPeaks = signal.find_peaks(
        #     accright[run][1, :],
        #     height=np.max(accright[1, :]) * .5,
        #     distance=fs,  # 1 s
        # )
        # ax.scatter(posPeaks[0] / fs, posPeaks[1]['peak_heights'], c='gray')
        
        
    # make plots pretty
    ax.set_xlabel('Time (sec)', size=20)
    ax.tick_params(axis='both', which='both', labelsize=16)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    
    # helping lines for alforithm finetuning
    # ax.axhline(np.median(spos), label='median')
    # ax.axhline(np.percentile(spos, 75), label='75-%', ls='dotted')
    # ax.axhline(-np.percentile(spos, 75), label='-75-%', color='red', ls='dotted')

# make plot pretty
for ax in [ax0, ax1]: ax.set_ylabel('Acceleration (m/s/s)', size=20)
ax2.spines['left'].set_visible(False)

# define zoomed subplots
ax1.set_xlim(ax1_x1, ax1_x2)
ax2.set_xlim(ax2_x1, ax2_x2)

# get rid off duplicate legend labels
handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
fig.legend(
    by_label.values(), by_label.keys(),
    fontsize=20, frameon=False, ncol=3,
    loc='upper left', bbox_to_anchor=(.03, -.01)
)
plt.suptitle(f'sub008: {run}', x=.8, y=.99,
    color='gray', alpha=.8, fontsize=16,)
# plt.tight_layout(pad=.1)

plt.savefig(os.path.join(plot_fdir, plot_fname),
    dpi=150, facecolor='white', bbox_inches='tight')

plt.show()

In [ ]:
### PLOTTING ###
## using xyz input and output-indices of pausedTap function
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

ax.plot(xEpoch, label='x', alpha=.5, c='blue')
ax.plot(yEpoch, label='y', alpha=.8, c='green')
ax.plot(zEpoch, label='z', alpha=.5, c='orange')
# plot vertical lines at tap-moments
for t, tap in enumerate(tapi):
    if t == 0:
        ax.axvline(tap[0], c='green', ls='dotted', alpha=.5, label='start UP')
        ax.axvline(tap[1], c='green', ls='solid', alpha=.5, label='end UP')
        ax.axvline(tap[2], c='purple', ls='dotted', alpha=.5, label='start DOWN')
        ax.axvline(tap[3], c='purple', ls='solid', alpha=.5, label='end DOWN')
    else:
        ax.axvline(tap[0], c='green', ls='dotted', alpha=.5,)
        ax.axvline(tap[1], c='green', ls='solid', alpha=.5,)
        ax.axvline(tap[2], c='purple', ls='dotted', alpha=.5,)
        ax.axvline(tap[3], c='purple', ls='solid', alpha=.5,)

peaklabel = ['lowPos', 'highPos', 'lowNeg', 'highNeg']
peakcol = ['lightgreen', 'green', 'orange', 'r']
for p, peaks in enumerate([smallPos, largePos, smallNeg, largeNeg]):
    ax.scatter(
        peaks[0],
        yEpoch[peaks[0]],
        label=peaklabel[p],
        color=peakcol[p],
        s=50,
    )

for mov in movei:  # check saved otherMovements
    ax.axvline(mov[0], color='gray', lw=1, ls='dotted')
    ax.axvline(mov[-1], color='gray', lw=1)

# ax.axhline(-.5e-7)
ax.axhline(posThr)
ax.axhline(0, c='gray', lw=.5,)
# determine what to show
ax.set_ylim(-2.5e-6, 2.5e-6)
# ax.set_xlim(istart, istop)
ax.set_xlim(0, 30000)
# ax.set_xticks(np.arange(18600, 20001, 200))
# ax.set_xticklabels(np.arange(18600, 20001, 200) // 200)
xticks = np.arange(0, len(yEpoch), 6000)
xlabs = np.arange(0, len(yEpoch) / 12000, .5)
ax.set_xticks(xticks)
ax.set_xticklabels(xlabs)

ax.set_ylabel('Raw Acceleration (m/s/s)', size=14)
ax.set_xlabel('Time (minutes)', size=14)

ax.set_title(run)

# ax.legend(frameon=False, fontsize=14, loc='upper right')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Plot Rest-moments
for idx in resti:
    ax.fill_betweenx(
        y=np.linspace(-3e-6, 3e-6, 5), x1=idx[0], x2=idx[1],
        color='blue', alpha=.1,)

if to_save:
    plt.savefig(
        os.path.join(temp_save, 'ACC', f'mountFingerTap_sub08_{run[-6:]}_2'),
        dpi=300, facecolor='w',
    )
plt.show()

print(f'{run} finisihed')